In [9]:
!rm -rf yolov5_RS
!git clone https://github.com/oballinger/yolov5_RS  # clone repo
%cd /kaggle/working/yolov5_RS
!pip install -qr requirements.txt && pip install -q roboflow


import torch # install pytorch
import os # for os related operations
from IPython.display import Image, clear_output  # to display images

Cloning into 'yolov5_RS'...
remote: Enumerating objects: 11622, done.
remote: Total 11622 (delta 0), reused 0 (delta 0), pack-reused 11622 (from 1)
Receiving objects: 100% (11622/11622), 77.33 MiB | 21.69 MiB/s, done.
Resolving deltas: 100% (7689/7689), done.
/kaggle/working/yolov5_RS


In [ ]:
!pip install geemap  -q

In [82]:
import pandas as pd
import ee
#import geemap.foliumap as geemap
import requests
from PIL import Image
from PIL import ImageDraw
from io import BytesIO
import torch
import PIL

In [48]:
#!earthengine authenticate
ee.Authenticate()
print('Auth')
ee.Initialize()

Auth


In [49]:
def detect(input, visParams, weight, labels=True):

  print('detecting')
  # Get the AOI from the map
  aoi = ee.FeatureCollection(Map.draw_features)
  mapScale=Map.getScale()


  # Visualize the raster in Earth Engine and get a download URL
  image_url=input.visualize(bands=visParams['bands'], max=visParams['max']).getThumbURL({"region":aoi.geometry(), 'scale':mapScale})


  # Load the image into a PIL image
  response = requests.get(image_url)
  img = Image.open(BytesIO(response.content))


  # Load the model
  model =torch.hub.load('.','custom', path='weights/{}.pt'.format(weight),source='local',_verbose=False)
  
  # Run inference
  results = model(img)


  # Count the number of detections
  counts=pd.DataFrame(results.pandas().xyxy[0].groupby('name').size()).reset_index().rename(columns={0:'count','name':'detected'}).set_index('count')


  # Display the results
  results.show(labels=labels)


  # Print the number of detections and the date of the image
  print(ee.Date(input.get('system:time_start')).format("dd-MM-yyyy").getInfo())
  print(counts)
  
  return counts

In [45]:
!jupyter nbextension enable --py --sys-prefix ipyleaflet
!pip install ipyleaflet
!jupyter nbextension install --py --sys-prefix ipyleaflet
!jupyter nbextension enable --py --sys-prefix ipyleaflet


Traceback (most recent call last):
  File "/opt/conda/bin/jupyter-nbextension", line 10, in <module>
    sys.exit(main())
  File "/opt/conda/lib/python3.10/site-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.10/site-packages/notebook/nbextensions.py", line 972, in start
    super().start()
  File "/opt/conda/lib/python3.10/site-packages/jupyter_core/application.py", line 270, in start
    self.subapp.start()
  File "/opt/conda/lib/python3.10/site-packages/notebook/nbextensions.py", line 882, in start
    self.toggle_nbextension_python(self.extra_args[0])
  File "/opt/conda/lib/python3.10/site-packages/notebook/nbextensions.py", line 855, in toggle_nbextension_python
    return toggle(module,
  File "/opt/conda/lib/python3.10/site-packages/notebook/nbextensions.

In [79]:
# load the past 10 years of NAIP imagery
naip = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date('2012-01-01', '2022-01-01'))
import geemap as gmap

# set some thresholds
trueColorVis = {
  'bands':['R', 'G', 'B'],
  'min': 0,
  'max': 300,
};
# initialize our map
Map = geemap.Map()
Map.setCenter(-110.84,32.16,17)
naip.first()
Map.addLayer(naip.first(), trueColorVis, "naip", True)
Map
#naip.first()

In [83]:
# perp
import ee

# Initialize Earth Engine
ee.Initialize()

# Load NAIP imagery
naip = ee.ImageCollection('USDA/NAIP/DOQQ').filterBounds(ee.Geometry.Point(-110.84, 32.16)).filter(ee.Filter.date('2010-01-01', '2022-12-31'))

# Set visualization parameters
trueColorVis = {
    'bands': ['R', 'G', 'B'],
    'min': 0,
    'max': 255,
}

# Initialize the map
Map = geemap.Map()
Map.setCenter(-110.84, 32.16, 17)

# Add the first image from the collection to the map
if naip.size().getInfo() > 0:
    first_image = naip.first()
    Map.addLayer(first_image, trueColorVis, "NAIP", True)
else:
    print("No NAIP imagery found for the specified location and date range.")

# Display the map
Map

In [90]:
print('joe')
# Get the polygon we just drew on the map 
aoi=ee.FeatureCollection(Map.draw_features)
aoi = ee.Geometry.Polygon([
    [-110.85, 32.15],
    [-110.85, 32.17],
    [-110.83, 32.17],
    [-110.83, 32.15]
])
print(aoi)
print('jaoe')
print('a')
# Get a list of all the images in the collection
naip_list=naip.filterBounds(aoi).toList(naip.size())
print('joeb')


# Iterate through the list of images and run detection on each one)
for num in range(0,naip_list.size().getInfo()):
  print('pip')
  detection = detect(ee.Image(naip_list.get(num)), trueColorVis,'general',labels=False)
  print('joe')
  df=df.append(detection) # store the results in a dataframe
print('sam')

joe
ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -110.85,
              32.15
            ],
            [
              -110.85,
              32.17
            ],
            [
              -110.83,
              32.17
            ],
            [
              -110.83,
              32.15
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})
jaoe
a
joeb
pip
detecting


AttributeError: 'Map' object has no attribute 'getScale'

In [63]:
df

NameError: name 'df' is not defined